<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Description" data-toc-modified-id="Data-Description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Description</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Load-the-data" data-toc-modified-id="Load-the-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load the data</a></span></li><li><span><a href="#Feature-Selection" data-toc-modified-id="Feature-Selection-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Feature Selection</a></span></li><li><span><a href="#Train-test-split" data-toc-modified-id="Train-test-split-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Train test split</a></span></li><li><span><a href="#Scaling-and-making-pipeline" data-toc-modified-id="Scaling-and-making-pipeline-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Scaling and making pipeline</a></span></li><li><span><a href="#Log-transform-large-values" data-toc-modified-id="Log-transform-large-values-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Log transform large values</a></span></li></ul></div>

# Data Description

This dataset contains house sale prices for King County,
which includes Seattle.
It includes homes sold between May 2014 and May 2015.

- Dependent features: 1 (price)
- Features : 19 home features
- Id:  1 house ID

Task: Try to estimate the price based on given features.

![](../data/raw/data_description.png)

# Imports

In [1]:
# my custom module
import src
import bhishan
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(color_codes=True)

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import os
import time

# Jupyter notebook settings for pandas
pd.set_option('display.max_columns', 200)
# pd.set_option('display.float_format', '{:,.4f}'.format) # numbers sep by comma
pd.set_option('display.max_rows', 100) # None for all the rows
pd.set_option('display.max_colwidth', 200)

print([(x.__name__,x.__version__) for x in [np, pd,sns,matplotlib]])

[('numpy', '1.16.4'), ('pandas', '0.25.0'), ('seaborn', '0.9.0'), ('matplotlib', '3.1.1')]


In [3]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [4]:
import scipy
import sklearn

print([(x.__name__,x.__version__) for x in [scipy, sklearn]])

[('scipy', '1.3.0'), ('sklearn', '0.21.2')]


In [5]:
# scale and split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [6]:
# regressors
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [7]:
# regressor preprocessing
from sklearn.preprocessing import PolynomialFeatures

In [8]:
# pipeline
from sklearn.pipeline import Pipeline

In [9]:
# metrics
from sklearn import metrics
from sklearn.metrics import mean_squared_error

In [10]:
# cross validation
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [11]:
df_eval = pd.DataFrame({'Model': [],
                           'Details':[],
                           'Root Mean Squared Error (RMSE)':[],
                           'R-squared (training)':[],
                           'Adjusted R-squared (training)':[],
                           'R-squared (test)':[],
                           'Adjusted R-squared (test)':[],
                           '5-Fold Cross Validation':[]})

# Load the data

In [12]:
# I will just take column names from this and will use cleaned data further.
df_raw = pd.read_csv('../data/raw/kc_house_data.csv')
df_raw.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [13]:
df_clean = pd.read_csv('../data/processed/data_cleaned_encoded.csv')
print(df_clean.shape)
df_clean.head()

(21613, 92)


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,yr_sales,age,yr_renovated2,age_after_renovation,zipcode_top10,zipcode_houses,basement_bool,renovation_bool,age_cat,age_after_renovation_cat,waterfront_0,waterfront_1,view_0,view_1,view_2,view_3,view_4,condition_1,condition_2,condition_3,condition_4,condition_5,grade_1,grade_10,grade_11,grade_12,grade_13,grade_3,grade_4,grade_5,grade_6,grade_7,grade_8,grade_9,zipcode_top10_98004,zipcode_top10_98006,zipcode_top10_98033,zipcode_top10_98039,zipcode_top10_98040,zipcode_top10_98102,zipcode_top10_98105,zipcode_top10_98155,zipcode_top10_98177,zipcode_top10_others,age_cat_0,age_cat_1,age_cat_2,age_cat_3,age_cat_4,age_cat_5,age_cat_6,age_cat_7,age_cat_8,age_cat_9,age_after_renovation_cat_0,age_after_renovation_cat_1,age_after_renovation_cat_2,age_after_renovation_cat_3,age_after_renovation_cat_4,age_after_renovation_cat_5,age_after_renovation_cat_6,age_after_renovation_cat_7,age_after_renovation_cat_8,age_after_renovation_cat_9,log1p_price,log1p_sqft_living,log1p_sqft_lot,log1p_sqft_above,log1p_sqft_basement,log1p_sqft_living15,log1p_sqft_lot15
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,2014,59,1955,59,others,262,0,0,5,5,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,12.309987,7.074117,8.639588,7.074117,0.000000,7.201171,8.639588
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,2014,63,1991,23,others,410,1,1,5,2,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,13.195616,7.852050,8.887791,7.682943,5.993961,7.433075,8.941153
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062,2015,82,1933,82,others,283,0,0,7,7,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,12.100718,6.647688,9.210440,6.647688,0.000000,7.908755,8.995041
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,2014,49,1965,49,others,263,1,0,4,4,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,13.311331,7.581210,8.517393,6.957497,6.814543,7.215975,8.517393
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,2015,28,1987,28,others,441,0,0,2,2,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,13.142168,7.427144,8.997271,7.427144,0.000000,7.496097,8.923191


# Feature Selection

- Consider numeical columns: waterfront, view, condition, grade
- Binned columns: age and age_after_renovation
- Also take variables as numbers: zipcode, lat, long
- sqft_basement = sqft_living - sqft_above (so we can drop this column)
- Drop date columns and id column: id, date
- Optional features: yr_built, yr_renovated, yr_sales

In [14]:
df_raw.head(2)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639


In [15]:
df_raw.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [16]:
features_raw_all = ['bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15']

df = df_clean[features_raw_all + ['price']]
correlation = df.corr(method='pearson')

cols10 = correlation.nlargest(10, 'price').index
cols10

Index(['price', 'sqft_living', 'grade', 'sqft_above', 'sqft_living15',
       'bathrooms', 'view', 'sqft_basement', 'bedrooms', 'lat'],
      dtype='object')

# Train test split

In [17]:
X = df[cols10].drop('price',axis=1).values

y = df['price'].values

X_train, X_test, y_train, y_test = train_test_split (X,y,
                                                     test_size=0.20,
                                                     random_state=100)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((17290, 9), (17290,), (4323, 9), (4323,))

# Scaling and making pipeline

In [18]:
pipelines = []

pipelines.append(('ScaledLR', Pipeline([('Scaler', StandardScaler()),
                                        ('LR',LinearRegression())])) )


pipelines.append(('ScaledLASSO', Pipeline([('Scaler', StandardScaler()),
                                           ('LASSO', Lasso())])))

pipelines.append(('ScaledEN', Pipeline([('Scaler', StandardScaler()),
                                        ('EN', ElasticNet())])))

pipelines.append(('ScaledKNN', Pipeline([('Scaler', StandardScaler()),
                                         ('KNN', KNeighborsRegressor())])))

pipelines.append(('ScaledDT', Pipeline([('Scaler', StandardScaler())
                                          ,('DT', DecisionTreeRegressor())])))

pipelines.append(('ScaledGBM', Pipeline([('Scaler', StandardScaler()),
                                         ('GBM', GradientBoostingRegressor())])))

In [19]:
df_cv = pd.DataFrame({'Model': [],
                      '5-Fold Cross Validation Mean':[],
                      '5-Fold Cross Validation Std':[]
                     })

for name, model in pipelines:
    kfold = KFold(n_splits=10, random_state=100)
    cv_results = cross_val_score(model,
                                 X_train,
                                 y_train,
                                 cv=kfold,
                                 scoring='neg_mean_squared_error')

    
    df_cv.loc[len(df_cv)] = [name, cv_results.mean(), cv_results.std() ]

/Users/poudel/miniconda3/envs/dataSc/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 23763658211941.75, tolerance: 210041168862.76996
  positive)
/Users/poudel/miniconda3/envs/dataSc/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 23637244377506.688, tolerance: 209121000755.08066
  positive)
/Users/poudel/miniconda3/envs/dataSc/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 23615544489012.0, tolerance: 210523960928.1077
  positive)
/Users/poudel/miniconda3/envs/dataSc/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Obje

In [20]:
df_cv

,Model,5-Fold Cross Validation Mean,5-Fold Cross Validation Std
0,ScaledLR,-4.916246e+10,4.338010e+09
1,ScaledLASSO,-4.914959e+10,4.322910e+09
2,ScaledEN,-5.240598e+10,5.081617e+09
3,ScaledKNN,-3.655357e+10,4.327630e+09
4,ScaledDT,-5.350454e+10,6.419970e+09
5,ScaledGBM,-2.833421e+10,3.935830e+09


# Log transform large values

In [21]:
df_clean['log_price'] = np.log1p(df_clean['price'])
df_clean['log_sqft_living'] = np.log1p(df_clean['sqft_living'])
df_clean['log_sqft_living15'] = np.log1p(df_clean['sqft_living15'])
df_clean['log_sqft_lot'] = np.log1p(df_clean['sqft_lot'])
df_clean['log_sqft_lot15'] = np.log1p(df_clean['sqft_lot'])

In [22]:
features_raw_all = ['bedrooms', 'bathrooms', 'log_sqft_living',
       'log_sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'log_sqft_living15', 'log_sqft_lot15']

target = ['log_price']

df = df_clean[features_raw_all + target]
correlation = df.corr(method='pearson')

cols10 = correlation.nlargest(10, 'log_price').index
cols10

Index(['log_price', 'grade', 'log_sqft_living', 'log_sqft_living15',
       'sqft_above', 'bathrooms', 'lat', 'view', 'bedrooms', 'sqft_basement'],
      dtype='object')

In [23]:
X = df[cols10].drop('log_price',axis=1).values

y = df['log_price'].values

X_train, X_test, y_train, y_test = train_test_split (X,y,
                                                     test_size=0.20,
                                                     random_state=100)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((17290, 9), (17290,), (4323, 9), (4323,))

In [24]:
df_cv = pd.DataFrame({'Model': [],
                      '5-Fold Cross Validation Mean':[],
                      '5-Fold Cross Validation Std':[]
                     })

for name, model in pipelines:
    kfold = KFold(n_splits=10, random_state=100)
    cv_results = cross_val_score(model, X_train, y_train,
                                 cv=kfold, scoring='neg_mean_squared_error')

    
    df_cv.loc[len(df_cv)] = [name, cv_results.mean(), cv_results.std() ]

In [25]:
df_cv.sort_values(['5-Fold Cross Validation Mean'])

,Model,5-Fold Cross Validation Mean,5-Fold Cross Validation Std
1,ScaledLASSO,-0.276666,0.010241
2,ScaledEN,-0.276666,0.010241
4,ScaledDT,-0.088682,0.003948
0,ScaledLR,-0.074869,0.002270
3,ScaledKNN,-0.057896,0.002748
5,ScaledGBM,-0.046152,0.002205


In [26]:
# scaled gradient boosted model performs best.

In [27]:
from sklearn.model_selection import GridSearchCV

scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)

param_grid = dict(n_estimators=np.array([50,100,200,300,400]))

model = GradientBoostingRegressor(random_state=100)
kfold = KFold(n_splits=10, random_state=100)

grid = GridSearchCV(estimator=model,
                    param_grid=param_grid,
                    scoring='neg_mean_squared_error',
                    cv=kfold)

grid_result = grid.fit(rescaledX, y_train)

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

In [28]:
df_grid_gbm = pd.DataFrame({'n_estimators': params,
                           'mean': means,
                           'std': stds})


pd.set_option('display.float_format', '{:,.10f}'.format)
df_grid_gbm.sort_values('mean')

,n_estimators,mean,std
0,{'n_estimators': 50},-0.0481284830,0.0022335795
1,{'n_estimators': 100},-0.0461452888,0.0021919060
2,{'n_estimators': 200},-0.0452854724,0.0020936393
3,{'n_estimators': 300},-0.0449544950,0.0020545709
4,{'n_estimators': 400},-0.0448699076,0.0020616437


In [29]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: -0.044870 using {'n_estimators': 400}


In [30]:
from sklearn.metrics import mean_squared_error

scaler = StandardScaler().fit(X_train)
rescaled_X_train = scaler.transform(X_train)

model = GradientBoostingRegressor(random_state=100, n_estimators=400)
model.fit(rescaled_X_train, y_train)

# transform the validation dataset
rescaled_X_test = scaler.transform(X_test)
predictions = model.predict(rescaled_X_test)
print ('mse = ', mean_squared_error(y_test, predictions))

mse =  0.04408488121499348


In [31]:
nmse = model.score(X_test, y_test)
print('negative mse = ', nmse)

negative mse =  -4.737650039465197


In [32]:
actual_y_test = np.expm1(y_test)
actual_predicted = np.expm1(predictions)

diff = abs(actual_y_test - actual_predicted)

df_preds = pd.DataFrame({'y_test': actual_y_test,
                        'y_pred' : actual_predicted,
                        'difference' : diff})
df_preds = df_preds.astype(int)
df_preds.head(5)

,y_test,y_pred,difference
0,284999,347409,62409
1,239950,234929,5020
2,460000,463112,3112
3,397499,471384,73884
4,544999,555766,10766


In [33]:
r2 = sklearn.metrics.r2_score(df_preds['y_test'],
                              df_preds['y_pred'])
r2

0.7764399077500295

In [34]:
def adjustedR2(rsquared,nrows,kcols):
    return rsquared- (kcols-1)/(nrows-kcols) * (1-rsquared)

In [35]:
ar2 = adjustedR2(r2, X_test.shape[0], X_test.shape[1])
print('adjustedr2 = ', ar2)

adjustedr2 =  0.7760253317792367
